In [18]:
!sudo -H pip install -q transformers --upgrade

In [2]:
!sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

     |████████████████████████████████| 20.0MB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 460kB 6.2MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 235kB 17.4MB/s eta 0:00:01
     |████████████████████████████████| 389kB 21.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 37.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 35.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 26.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 30.8MB/s eta 0:00:01


     |████████████████████████████████| 1.5MB 36.9MB/s eta 0:00:01
     |████████████████████████████████| 8.0MB 64.3MB/s eta 0:00:01
     |████████████████████████████████| 204kB 106.8MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 101.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 30.6MB/s eta 0:00:01


     |████████████████████████████████| 471kB 108.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 48.0MB/s  eta 0:00:01
  Created wheel for lm-eval: filename=lm_eval-0.4.0-cp38-none-any.whl size=994996 sha256=95b8fa3665066c6f5e9b265fd92977b8bd17c5705244f77bc0a3e04dc752ed54
  Stored in directory: /root/.cache/pip/wheels/60/08/c8/cc3a820545d8c8990dc313dfd01c0ee280999cd9e796973e35
Successfully built lm-eval
  Created wheel for rouge-score: filename=rouge_score-0.1.2-cp38-none-any.whl size=24937 sha256=d690c8766954a6e422dbc395872d759251136ff2f5a486812eb13f63602fe184
  Stored in directory: /root/.cache/pip/wheels/df/aa/59/74f33db3bbedf322bcaadca4a43750ea5eb523bbe742d78b25
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-cp38-none-any.whl size=16865 sha256=4bf2e70434852c52511a33fafe78e80af6c7ed63bdaa61af18fdd01ce385239b
  Stored in directory: /root/.cache/pip/wheels/2e/10/02/9725042cf8a987b45e38bcfb106f4b0cdcc9d8968e248b93e1
Successfully built rouge-score sqlitedict
E

In [1]:
import pandas

In [19]:
!pip install -U datasets

  Using cached https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c2/a6/c5cb599d917904878f220a4dbdfdcc4ef291dd3956c35b3b0dc6fc42fb6d/multiprocess-0.70.15-py38-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a7/7a/b8d5b40870349e468b08e1ecb65a500b0ed1fe4937cf6bda01e7b85c8cb4/pyarrow-14.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c8/6b/6600ac24725c7388255b2f5add93f91e58a5d7efaf4af244fdbcc11a541b/PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-

In [1]:
import numpy

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [3]:
!pip list | grep cuda

nvidia-cuda-cupti-cu11   11.7.101    
nvidia-cuda-nvrtc-cu11   11.7.99     
nvidia-cuda-runtime-cu11 11.7.99     


In [4]:
import pandas

In [5]:
import torch
torch.__version__

'2.0.1+cu117'

In [6]:
model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [7]:
quant_path = '/data/mistral/query-to-mql/exp-10/nov-20/quant-model'

In [8]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [9]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [11]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

import torch
torch.cuda.empty_cache()

import gc
gc.collect()

31

In [12]:
print(os.environ["PYTORCH_CUDA_ALLOC_CONF"])

max_split_size_mb:512


In [ ]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating validation split: 0 examples [00:00, ? examples/s]

AWQ:   3%|▎         | 1/32 [01:14<38:31, 74.55s/it]

In [15]:
#model.save_quantized(quant_path)
#tokenizer.save_pretrained(quant_path)

('/data/quantization-trials/quant-model/tokenizer_config.json',
 '/data/quantization-trials/quant-model/special_tokens_map.json',
 '/data/quantization-trials/quant-model/vocab.json',
 '/data/quantization-trials/quant-model/merges.txt',
 '/data/quantization-trials/quant-model/added_tokens.json',
 '/data/quantization-trials/quant-model/tokenizer.json')

### Inference on quantized model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [14]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [15]:
import time

In [23]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 21.147445678710938


In [24]:
print(output1)

</s>what is art?

Art is a way of expressing one's self. It is a way of expressing one's thoughts, feelings, and ideas. It is a way of expressing one's self through the use of various media.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art for

### Inference on un-quantized model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
model_path = '/data/quantization-trials/merged-model'

In [20]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

time taken is : 23.784337282180786


In [26]:
print(output2)

</s>what is art?

Art is a creative expression of the human mind. It is a way of expressing the human mind through the use of various media.

The word art is derived from the Latin word artus, which means "to make". The word art is used in many different contexts, including:

- to describe the beauty of a work of art

- to describe the meaning of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work 

In [26]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [27]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out